In [1]:
library(tidyverse)
library(here)
library(glue)
library(furrr)
suppressPackageStartupMessages(library(rtracklayer))
devtools::load_all(".")

plan(multiprocess, workers = 20)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.2.1     ✔ purrr   0.3.2
✔ tibble  2.1.3     ✔ dplyr   0.8.3
✔ tidyr   1.0.0     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
here() starts at /mnt/expressions/mp/archaic-ychr

Attaching package: ‘glue’

The following object is masked from ‘package:dplyr’:

    collapse

Loading required package: future
Loading ychr


In [2]:
samples <- list.files(here("data/vcf"), "full_.*.vcf.gz$") %>%
    str_subset("mez2_dp|modern|merged", negate = T) %>%
    str_replace("full_(.*).vcf.gz", "\\1")

In [3]:
samples

[1] "a00_1"             "a00_2"             "a00"              
 [4] "chimp"             "den_snpad"         "den"              
 [7] "den4"              "den8"              "mez2_snpad"       
[10] "mez2"              "S_BedouinB-1"      "S_Burmese-1"      
[13] "S_Dai-2"           "S_Dinka-1"         "S_Finnish-2"      
[16] "S_French-1"        "S_Gambian-1"       "S_Han-2"          
[19] "S_Ju_hoan_North-1" "S_Karitiana-1"     "S_Mandenka-1"     
[22] "S_Mbuti-1"         "S_Papuan-2"        "S_Punjabi-1"      
[25] "S_Saami-2"         "S_Sardinian-1"     "S_Thai-1"         
[28] "S_Turkish-1"       "S_Yoruba-2"        "shotgun_mez2"     
[31] "shotgun_spy1"      "spy1"              "ustishim"

In [4]:
capture_bed <- here("data/coord/capture_full.bed")

In [5]:
capture_full <- import.bed(capture_bed)

In [6]:
sum(width(capture_full))

[1] 6913053

In [7]:
full_counts <- future_map_dfr(samples, function(ind) {
    gt_all <- read_vcf(here(glue("data/vcf/full_{ind}.vcf.gz")), mindp = 3, maxdp = 0.98, bed_filter = capture_bed)
    gt_nodmg <- read_vcf(here(glue("data/vcf/full_{ind}.vcf.gz")), mindp = 3, maxdp = 0.98, tv_only = T, bed_filter = capture_bed)

    tibble(
        individual = ind,
        total = sum(width(capture_full)),
        all = nrow(gt_all),
        all_prop = all / total,
        nodmg = nrow(gt_nodmg),
        nodmg_prop = nodmg / total
    )
})

In [19]:
full_counts %>%
    arrange(all) %>%
    mutate_at(vars(contains("prop")), ~ sprintf("%.2f%%", 100 * .)) %>%
    select(-total)

individual,all,all_prop,nodmg,nodmg_prop
<chr>,<int>,<chr>,<int>,<chr>
shotgun_spy1,2664973,38.55%,2636589,38.14%
spy1,3502527,50.67%,3470817,50.21%
shotgun_mez2,3822247,55.29%,3762906,54.43%
den4,4731419,68.44%,4705977,68.07%
den8,5851442,84.64%,5828466,84.31%
den,6073540,87.86%,6062391,87.69%
mez2,6349143,91.84%,6346879,91.81%
chimp,6453350,93.35%,6391512,92.46%
den_snpad,6540959,94.62%,6528856,94.44%
